# Steane state preparation fault tolerance check

Goal: Validate that the following two diagrams are fault equivalent:

<center >
    <img src="steane-state-prep-state.png" style="height: 300px">
    <img src="steane-state-prep-circuit.png" style="height: 300px">
</center>

### Diagram Preparation

In [ ]:
from paritea import draw
from paritea.diagram import Diagram, NodeType

d_state = Diagram()
xs = [d_state.add_node(NodeType.X, x=0, y=(7 / 2) + i - (4 / 2)) for i in range(4)]
zs = [d_state.add_node(NodeType.Z, x=2, y=i) for i in range(7)]
bs = [d_state.add_node(NodeType.B, x=3, y=i) for i in range(7)]
d_state.add_edges([(xs[0], zs[0]), (xs[0], zs[1]), (xs[0], zs[2])])
d_state.add_edges([(xs[1], zs[0]), (xs[1], zs[2]), (xs[1], zs[3]), (xs[1], zs[5])])
d_state.add_edges([(xs[2], zs[2]), (xs[2], zs[4]), (xs[2], zs[5])])
d_state.add_edges([(xs[3], zs[0]), (xs[3], zs[5]), (xs[3], zs[6])])
d_state.add_edges([(z, b) for z, b in zip(zs, bs)])
d_state.set_io(inputs=[], outputs=bs, virtual=False)

draw(d_state)

In [ ]:
d_circuit = Diagram()

# First layer
qubits = [
    d_circuit.add_node(NodeType.X, x=0, y=0),
    d_circuit.add_node(NodeType.Z, x=0, y=1),
    d_circuit.add_node(NodeType.Z, x=0, y=2),
    d_circuit.add_node(NodeType.Z, x=0, y=3),
    d_circuit.add_node(NodeType.X, x=0, y=4),
    d_circuit.add_node(NodeType.X, x=0, y=5),
    d_circuit.add_node(NodeType.X, x=0, y=6),
    d_circuit.add_node(NodeType.X, x=0, y=7), # Extra auxiliary qubit
]

# More layers
def add_cnot(*, ct: int, tgt: int, x: int):
    ct_node, tgt_node = d_circuit.add_node(NodeType.Z, x=x, y=ct), d_circuit.add_node(NodeType.X, x=x, y=tgt)
    d_circuit.add_edges([(ct_node, tgt_node), (ct_node, qubits[ct]), (tgt_node, qubits[tgt])])
    qubits[ct], qubits[tgt] = ct_node, tgt_node

add_cnot(ct=1, tgt=0, x=1)
add_cnot(ct=3, tgt=5, x=1)
add_cnot(ct=2, tgt=6, x=2)
add_cnot(ct=1, tgt=4, x=3)
add_cnot(ct=2, tgt=0, x=4)
add_cnot(ct=3, tgt=6, x=4)
add_cnot(ct=1, tgt=5, x=5)
add_cnot(ct=6, tgt=4, x=6)
add_cnot(ct=0, tgt=7, x=7)
add_cnot(ct=5, tgt=7, x=8)
add_cnot(ct=6, tgt=7, x=9)

# Final layer: Boundaries for the first 7 qubits, X basis measurement for the auxiliary qubit
bs = [d_circuit.add_node(NodeType.B, x=11, y=i) for i in range(7)]
d_circuit.add_edges([(q, b) for q, b in zip(qubits, bs)])
d_circuit.add_edge(qubits[7], d_circuit.add_node(NodeType.X, x=10, y=7))
d_circuit.set_io(inputs=[], outputs=bs, virtual=False)

draw(d_circuit)

### Compute Pauli webs

In [ ]:
from paritea.web import compute_pauli_webs

draw_webs = False

stabilisers, regions = compute_pauli_webs(d_state)
print(f"Found {len(stabilisers)} stabilisers and {len(regions)} regions for d_state!")
for stabiliser in stabilisers:
    if draw_webs:
        draw(d_state, web=stabiliser)
assert len(regions) == 0

stabilisers, regions = compute_pauli_webs(d_circuit)
print(f"Found {len(stabilisers)} stabilisers and {len(regions)} regions for d_circuit!")
for stabiliser in stabilisers:
    if draw_webs:
        draw(d_circuit, web=stabiliser)
for region in regions:
    if draw_webs:
        draw(d_circuit, web=region)

### Noise model preparation

In [ ]:
from paritea.noise import NoiseModel

nm_state = NoiseModel.weighted_edge_flip_noise(d_state, w_x=1, w_y=2, w_z=1, idealised_edges=[e for x in xs for e in d_state.incident_edges(x)])
nm_circuit = NoiseModel.weighted_edge_flip_noise(d_circuit, w_x=1, w_y=2, w_z=1, idealised_edges=[])

### Check fault equivalence

In [ ]:
from paritea.equivalence import is_fault_equivalence

is_fault_equivalence(nm_state, nm_circuit, quiet=False)